In [ ]:
from utils import *
from plotly.subplots import make_subplots
from tqdm.auto import tqdm
import json
from loaders import *

In [ ]:
res_df, scalars = data_aug_v2_loader()
res_b_df, scalars_b = baseline_loader()
res_s_df, scalars_s = data_aug_sweep_loader()

In [ ]:
envs = res_df["env"].unique()

fig = make_subplots(
    cols=len(envs),
    column_titles=[*envs],
)

colors = make_color_iter()
colors = [next(colors) for _ in range(2)]
axis = 1

for col, env in enumerate(envs, 1):
    df1 = res_df[res_df["env"] == env].copy()
    df1["tag"] = df1["ratio"].apply(lambda r: f"K = {r}")
    df1 = df1.sort_values(by="tag")
    fig.add_trace(
        go.Box(
            x=df1["tag"],
            y=df1["score"],
            sizemode="sd",
            boxmean=True,
            name="Data-Augmented",
            offsetgroup="0",
            legendgroup="0",
            showlegend=(col == 1),
            marker=dict(color=colors[0]),
        ),
        row=1,
        col=col,
    )

    df2 = res_b_df[res_b_df["env"] == env].copy()
    df2 = df2[df2["ratio"].isin(df1["ratio"].unique())]
    df2["tag"] = df2["ratio"].apply(lambda r: f"K = {r}")
    df2 = df2.sort_values(by="tag")
    fig.add_trace(
        go.Box(
            x=df2["tag"],
            y=df2["score"],
            sizemode="sd",
            boxmean=True,
            name="Baseline",
            offsetgroup="1",
            legendgroup="1",
            showlegend=(col == 1),
            marker=dict(color=colors[1]),
        ),
        row=1,
        col=col,
    )

    fig.update_layout(
        **{f"xaxis{axis}": dict(title="Configuration")},
        **({f"yaxis{axis}": dict(title="Score")} if col == 1 else {}),
    )
    axis += 1

fig.update_layout(
    boxmode="group",
    width=1000,
    height=400,
)
fig.write_image("../tex/assets/data_aug.pdf")
fig

In [ ]:
envs = res_df["env"].unique()

fig = make_subplots(
    cols=len(envs),
    column_titles=[*envs],
)

colors = make_color_iter()
colors = [next(colors) for _ in range(2)]


def get_final(scalars, df, key):
    values = []
    for _, row in df.iterrows():
        test = scalars.read(row["path"])
        values.append(test[test["tag"] == key].iloc[-1]["value"].item())
    return values


axis = 1
for col, env in enumerate(envs, 1):
    df1 = res_df[res_df["env"] == env].copy()
    df1["tag"] = df1["ratio"].apply(lambda r: f"K = {r}")
    df1["val_loss"] = get_final(scalars, df1, "val/wm_loss")
    df1 = df1.sort_values(by="tag")
    fig.add_trace(
        go.Box(
            x=df1["tag"],
            y=df1["val_loss"],
            sizemode="sd",
            boxmean=True,
            name="Data-Augmented",
            offsetgroup="0",
            legendgroup="0",
            showlegend=(col == 1),
            marker=dict(color=colors[0]),
        ),
        row=1,
        col=col,
    )

    df2 = res_b_df[res_b_df["env"] == env].copy()
    df2 = df2[df2["ratio"].isin(df1["ratio"].unique())]
    df2["tag"] = df2["ratio"].apply(lambda r: f"K = {r}")
    df2["val_loss"] = get_final(scalars_b, df2, "val/wm_loss")
    df2 = df2.sort_values(by="tag")
    fig.add_trace(
        go.Box(
            x=df2["tag"],
            y=df2["val_loss"],
            sizemode="sd",
            boxmean=True,
            name="Baseline",
            offsetgroup="1",
            legendgroup="1",
            showlegend=(col == 1),
            marker=dict(color=colors[1]),
        ),
        row=1,
        col=col,
    )

    fig.update_layout(
        **{f"xaxis{axis}": dict(title="Configuration")},
        **({f"yaxis{axis}": dict(title="Model val loss")} if col == 1 else {}),
    )
    axis += 1

fig.update_layout(
    boxmode="group",
    width=1000,
    height=400,
)
fig.write_image("../tex/assets/data_aug.val_loss.pdf")
fig

In [ ]:
from itertools import islice

envs = sorted([*res_df["env"].unique()])
ratios = sorted([*res_df["ratio"].unique()])

specs = [[{} for _ in envs] for _ in ratios]

fig = make_subplots(
    cols=len(envs),
    column_titles=[*envs],
    rows=len(ratios),
    row_titles=[f"K = {r}" for r in ratios],
    horizontal_spacing=0.025,
    vertical_spacing=0.05,
    specs=specs,
)

val_ranges = {
    "Assault": [1.5, 6.0],
    "CrazyClimber": [0.5, 3.0],
    "MsPacman": [1.5, 4.0],
}

color_iter = make_color_iter()
colors = {"Base": next(color_iter), "Data Aug": next(color_iter)}


def add_line(test_dfs, scalars, key, row, col, **kwargs):
    dfs = []
    for _, test in test_dfs.iterrows():
        test_df = scalars.read(test["path"])
        test_df = test_df[test_df["tag"] == key].copy()
        dfs.append(test_df)
    dfs = pd.concat(dfs)
    g = dfs.groupby("step")
    df = pd.concat({"value": g["value"].mean()})
    df = df.reset_index()
    ratio = test_dfs.iloc[0]["ratio"]
    df["step"] *= ratio
    fig.add_trace(
        go.Scatter(
            x=df["step"],
            y=exp_mov_avg(df["value"], 1.0 - 5e-3 * ratio),
            **kwargs,
        ),
        row=row,
        col=col,
    )


axis = 1
# Data-augmented
for row, ratio in enumerate(ratios, 1):
    for col, env in enumerate(envs, 1):
        aug_tests = res_df[(res_df["env"] == env) & (res_df["ratio"] == ratio)]
        add_line(
            aug_tests,
            scalars,
            "wm/val_loss",
            row,
            col,
            name=f"Data Aug",
            legendgroup="Data Aug",
            showlegend=(row == 1 and col == 1),
            marker=dict(color=colors["Data Aug"]),
        )

        base_tests = res_b_df[(res_b_df["env"] == env) & (res_b_df["ratio"] == ratio)]
        add_line(
            base_tests,
            scalars_b,
            "wm/val_loss",
            row,
            col,
            name=f"Base",
            legendgroup="Base",
            showlegend=(row == 1 and col == 1),
            marker=dict(color=colors["Base"]),
        )

        min_, max_ = val_ranges[env]

        layout = {}
        if row == 2:
            layout[f"xaxis{axis}_title"] = "Env step"
        if col == 1:
            layout[f"yaxis{axis}_title"] = "Model val loss"
        layout = {
            **layout,
            f"yaxis{axis}": dict(
                # range=[min_, max_],
                type="log",
                range=[np.log10(min_), np.log10(max_)],
            ),
        }
        fig.update_layout(**layout)
        axis += 1


fig.update_layout(width=1000, height=500)
fig.write_image("../tex/assets/data_aug.val_curves.pdf")
fig

In [ ]:
from itertools import islice

envs = sorted([*res_df["env"].unique()])
ratios = sorted([*res_df["ratio"].unique()])

specs = [[{} for _ in envs] for _ in ratios]

fig = make_subplots(
    cols=len(envs),
    column_titles=[*envs],
    rows=len(ratios),
    row_titles=[f"K = {r}" for r in ratios],
    horizontal_spacing=0.025,
    vertical_spacing=0.05,
    specs=specs,
)

val_ranges = {
    "Assault": [0.3, 4.0],
    "CrazyClimber": [0.1, 3.0],
    "MsPacman": [0.25, 4.0],
}

color_iter = make_color_iter()
colors = {"Base": next(color_iter), "Data Aug": next(color_iter)}


def add_line(test_dfs, scalars, key, row, col, **kwargs):
    dfs = []
    for _, test in test_dfs.iterrows():
        test_df = scalars.read(test["path"])
        test_df = test_df[test_df["tag"] == key].copy()
        dfs.append(test_df)
    dfs = pd.concat(dfs)
    g = dfs.groupby("step")
    df = pd.concat({"value": g["value"].mean()})
    df = df.reset_index()
    ratio = test_dfs.iloc[0]["ratio"]
    df["step"] *= ratio
    fig.add_trace(
        go.Scatter(
            x=df["step"],
            y=exp_mov_avg(df["value"], 1.0 - 5e-3 * ratio),
            **kwargs,
        ),
        row=row,
        col=col,
    )


axis = 1
# Data-augmented
for row, ratio in enumerate(ratios, 1):
    for col, env in enumerate(envs, 1):
        aug_tests = res_df[(res_df["env"] == env) & (res_df["ratio"] == ratio)]
        add_line(
            aug_tests,
            scalars,
            "wm/loss",
            row,
            col,
            name=f"Data Aug",
            legendgroup="Data Aug",
            showlegend=(row == 1 and col == 1),
            marker=dict(color=colors["Data Aug"]),
        )
        # add_line(aug_tests, scalars, "wm/val_loss", row, col, name=f"Val DA")

        base_tests = res_b_df[(res_b_df["env"] == env) & (res_b_df["ratio"] == ratio)]
        add_line(
            base_tests,
            scalars_b,
            "wm/loss",
            row,
            col,
            name=f"Base",
            legendgroup="Base",
            showlegend=(row == 1 and col == 1),
            marker=dict(color=colors["Base"]),
        )
        # add_line(base_tests, scalars_b, "wm/val_loss", row, col, name=f"Val Base")

        min_, max_ = val_ranges[env]

        layout = {}
        if row == 2:
            layout[f"xaxis{axis}_title"] = "Env step"
        if col == 1:
            layout[f"yaxis{axis}_title"] = "Model train loss"
        layout = {
            **layout,
            f"yaxis{axis}": dict(
                # range=[min_, max_],
                type="log",
                range=[np.log10(min_), np.log10(max_)],
            ),
        }
        fig.update_layout(**layout)
        axis += 1


fig.update_layout(width=1000, height=500)
fig.write_image("../tex/assets/data_aug.train_curves.pdf")
fig

In [ ]:
envs = res_s_df["env"].unique()

fig = make_subplots(
    rows=len(envs),
    row_titles=[*envs],
    cols=1,
)

for row, env in enumerate(envs, 1):
    df_ = res_s_df[res_s_df["env"] == env].copy()
    df_ = df_.sort_values(by="type")
    ratio = df_["ratio"].iloc[0]
    df2 = res_b_df[(res_b_df["env"] == env) & (res_b_df["ratio"] == ratio)].copy()
    df2["type"] = "none"
    df_ = pd.concat([df2, df_])

    fig.add_trace(
        go.Box(x=df_["type"], y=df_["score"], sizemode="sd", showlegend=False),
        row=row,
        col=1,
    )

# fig.update_traces(meanline_visible=True)
fig.update_layout(
    width=1000,
    height=800,
    xaxis3_title="Aug Type",
    yaxis_title="Score",
    yaxis2_title="Score",
    yaxis3_title="Score",
)

fig.write_image("../tex/assets/data_aug.sweep.pdf")
fig